# Laminar Flow Analysis

This notebook covers the laminar flow analysis workflow:

1. Generating laminar flow XPCS data (synthetic)
2. Detecting angular dependence in C2
3. Configuring laminar flow mode with anti-degeneracy
4. Comparing per-angle mode variants
5. Interpreting shear parameters

**Use laminar flow mode when:** your sample is in a Couette shear cell and you
observe an angular dependence in the two-time correlation matrix.

---

## 1. Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from homodyne.config import ConfigManager
from homodyne.optimization.nlsq import fit_nlsq_jax
from homodyne.utils.logging import get_logger, log_phase

logger = get_logger(__name__)

## 2. Generate Laminar Flow Data

The laminar flow model adds a sinc² factor that depends on azimuthal angle φ.

In [ ]:
rng = np.random.default_rng(seed=42)

# True parameters for laminar flow
TRUE = {
    "D0": 800.0,  # Å²/s
    "alpha": -0.4,
    "D_offset": 0.05,
    "gamma_dot_0": 2.0,  # s⁻¹ (shear rate)
    "beta": -0.2,  # Slight shear rate decrease
    "gamma_dot_offset": 0.01,
    "phi_0": 15.0,  # degrees (flow direction offset)
    "contrast": 0.12,
    "offset": 1.0,
}

# Geometry
q = 0.054  # Å⁻¹
h = 5000.0  # Å (0.5 mm gap)
n_t = 40
n_phi = 12
dt = 0.1
t = dt * np.arange(n_t)
phi_deg = np.linspace(0, 330, n_phi)  # 0° to 330° in 30° steps
phi_rad = np.deg2rad(phi_deg)
phi0_rad = np.deg2rad(TRUE["phi_0"])


def sinc(x):
    """Unnormalized sinc: sin(x) / x with limit 1 at x=0."""
    return np.where(np.abs(x) < 1e-10, 1.0, np.sin(x) / x)


c2 = np.zeros((n_phi, n_t, n_t))
for i_phi, phi in enumerate(phi_rad):
    for i_t1 in range(n_t):
        for i_t2 in range(i_t1, n_t):
            t1_val, t2_val = t[i_t1], t[i_t2]

            # Diffusion kernel
            J = TRUE["D0"] * (
                t2_val ** (TRUE["alpha"] + 1) - t1_val ** (TRUE["alpha"] + 1)
            ) / (TRUE["alpha"] + 1) + TRUE["D_offset"] * (t2_val - t1_val)

            # Shear strain kernel
            Gamma = TRUE["gamma_dot_0"] * (
                t2_val ** (TRUE["beta"] + 1) - t1_val ** (TRUE["beta"] + 1)
            ) / (TRUE["beta"] + 1) + TRUE["gamma_dot_offset"] * (t2_val - t1_val)

            # Model
            g1_sq = np.exp(-2 * q**2 * J)
            sinc_arg = 0.5 * q * h * Gamma * np.cos(phi - phi0_rad)
            sinc_sq = sinc(sinc_arg) ** 2
            val = TRUE["offset"] + TRUE["contrast"] * g1_sq * sinc_sq

            noise = 0.003 * rng.standard_normal()
            c2[i_phi, i_t1, i_t2] = val + noise
            c2[i_phi, i_t2, i_t1] = val + noise

data = {
    "c2_exp": c2,
    "t1": t,
    "t2": t,
    "phi_angles_list": phi_deg,
    "wavevector_q_list": np.array([q]),
    "sigma": 0.003 * np.ones_like(c2),
    "L": h,
    "dt": dt,
}

print(f"Data shape: {c2.shape}  (n_phi={n_phi}, n_t={n_t})")
print(f"q = {q} Å⁻¹,  h = {h:.0f} Å = {h / 1e4:.2f} µm")
print(
    f"Expected sinc arg (max lag): {0.5 * q * h * TRUE['gamma_dot_0'] * (t[-1] - t[0]):.2f}"
)

## 3. Angular Dependence Visualization

This is the key diagnostic for laminar flow.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Left: C2 heatmap for two representative angles
ax = axes[0]
# angle closest to phi_0 (maximum shear effect)
i_phi_max = np.argmin(np.abs(phi_deg - TRUE["phi_0"]))
# angle closest to phi_0 + 90° (minimum shear effect)
i_phi_min = np.argmin(np.abs(phi_deg - (TRUE["phi_0"] + 90)))

im = ax.pcolormesh(
    t, t, c2[i_phi_max].T, cmap="hot", vmin=0.97, vmax=1.15, shading="auto"
)
ax.set_xlabel("t1 (s)")
ax.set_ylabel("t2 (s)")
ax.set_title(f"C2 at φ = {phi_deg[i_phi_max]:.0f}° (near flow direction)")
plt.colorbar(im, ax=ax)

# Right: decorrelation curves for all angles
ax = axes[1]
colors = plt.cm.hsv(np.linspace(0, 1, n_phi, endpoint=False))
for i_phi, (phi_val, color) in enumerate(zip(phi_deg, colors)):
    lag_idx = 5
    n_lags = n_t - 1
    lag_times = t[1 : n_lags + 1] - t[0]
    c2_lag = [
        np.mean([c2[i_phi, k, k + i] for k in range(n_t - i)])
        for i in range(1, n_lags + 1)
    ]
    ax.plot(
        lag_times,
        c2_lag,
        color=color,
        alpha=0.8,
        linewidth=1.5,
        label=f"{phi_val:.0f}°" if i_phi % 3 == 0 else None,
    )

ax.set_xscale("log")
ax.set_xlabel("Lag time (s)")
ax.set_ylabel("⟨C2⟩ at lag")
ax.set_title("Angular dependence (spread = shear signal)")
ax.legend(ncol=2, fontsize=8)

plt.tight_layout()
display(fig)
plt.close(fig)

# Compute angular variance ratio
lag_idx = 3
c2_at_lag = np.array([c2[i_phi, 0, lag_idx] for i_phi in range(n_phi)])
ratio = c2_at_lag.std() / c2_at_lag.mean()
print(f"Angular variance ratio: {ratio:.4f}")
print(
    f"→ {'Significant angular dependence → laminar_flow mode' if ratio > 0.02 else 'Weak angular dependence'} "
)

## 4. Configure Laminar Flow Mode

In [ ]:
config_yaml = """
analysis_mode: "laminar_flow"

analyzer_parameters:
  dt: 0.1
  geometry:
    stator_rotor_gap: 5000.0   # Å (= 0.5 µm)

optimization:
  method: "nlsq"
  nlsq:
    anti_degeneracy:
      per_angle_mode: "auto"

initial_parameters:
  parameter_names: [D0, alpha, D_offset, gamma_dot_t0, beta, gamma_dot_t_offset, phi0]
  values: [500.0, -0.5, 0.1, 2.0, 0.0, 0.01, 0.0]
"""

with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
    f.write(config_yaml)
    config_path = f.name

config_flow = ConfigManager(config_path)
analysis_mode = config_flow.config.get("analysis_mode", "laminar_flow")
print(f"Analysis mode: {analysis_mode}")
print(f"Parameters: {list(config_flow.get_initial_parameters().keys())}")

In [ ]:
print("Running NLSQ laminar flow fit (with anti-degeneracy)...")
with log_phase("NLSQ laminar_flow"):
    result_flow = fit_nlsq_jax(data, config_flow)

# Physical parameters come after per-angle scaling (n_phi contrasts + n_phi offsets)
phys_offset = 2 * n_phi

print(f"\nStatus:     {result_flow.convergence_status}")
print(f"chi^2_nu:   {result_flow.reduced_chi_squared:.4f}")
print(f"Time:       {result_flow.execution_time:.2f} s")
print()
print(f"{'Param':<22} {'True':>10} {'Fitted':>10} {'Error':>10} {'Rel. %':>8}")
print("-" * 65)

phys_params = [
    "D0",
    "alpha",
    "D_offset",
    "gamma_dot_0",
    "beta",
    "gamma_dot_offset",
    "phi_0",
]
for i, name in enumerate(phys_params):
    tv = TRUE[name]
    fv = result_flow.parameters[phys_offset + i]
    fe = result_flow.uncertainties[phys_offset + i]
    rel = abs(fv - tv) / abs(tv) * 100 if tv != 0 else float("nan")
    print(f"{name:<22} {tv:>10.4g} {fv:>10.4g} {fe:>10.4g} {rel:>8.1f}%")

## 5. Compare Per-Angle Modes

Demonstrate the importance of anti-degeneracy by comparing modes.

In [ ]:
# Compare auto vs constant mode for D0 recovery
modes_to_test = ["auto", "constant"]
results_by_mode = {}

for mode in modes_to_test:
    config_yaml_mode = config_yaml.replace(
        'per_angle_mode: "auto"', f'per_angle_mode: "{mode}"'
    )
    with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
        f.write(config_yaml_mode)
        temp_path = f.name

    cfg = ConfigManager(temp_path)
    r = fit_nlsq_jax(data, cfg)
    results_by_mode[mode] = r
    os.unlink(temp_path)

print("Per-angle mode comparison:")
print(
    f"{'Mode':<12} {'D0 (true: 800)':>16} {'gamma_dot_0 (true: 2)':>22} {'chi2_nu':>10}"
)
print("-" * 65)
for mode, r in results_by_mode.items():
    D0_err = abs(r.parameters[phys_offset] - 800) / 800 * 100
    gd_err = abs(r.parameters[phys_offset + 3] - 2.0) / 2.0 * 100
    print(
        f"{mode:<12} {r.parameters[phys_offset]:>10.0f}  ({D0_err:+.1f}%)  "
        f"{r.parameters[phys_offset + 3]:>10.3f}  ({gd_err:+.1f}%)  "
        f"{r.reduced_chi_squared:>10.4f}"
    )

## 6. Angular Dependence Visualization

Visualize the sinc² pattern to confirm shear detection.

In [ ]:
# Plot C2 amplitude vs angle for fixed lag time
D0_fit = result_flow.parameters[phys_offset]
alpha_fit = result_flow.parameters[phys_offset + 1]
D_offset_fit = result_flow.parameters[phys_offset + 2]
gd0_fit = result_flow.parameters[phys_offset + 3]
beta_fit = result_flow.parameters[phys_offset + 4]
gd_offset_fit = result_flow.parameters[phys_offset + 5]
phi0_fit = result_flow.parameters[phys_offset + 6]

phi_fine = np.linspace(0, 360, 360)
phi_fine_rad = np.deg2rad(phi_fine)
phi0_fit_rad = np.deg2rad(phi0_fit)

# At lag time = 5 frames
lag_idx = 5
t1_val, t2_val = t[0], t[lag_idx]

J_fit = D0_fit * (t2_val ** (alpha_fit + 1) - t1_val ** (alpha_fit + 1)) / (
    alpha_fit + 1
) + D_offset_fit * (t2_val - t1_val)
Gamma_fit = gd0_fit * (t2_val ** (beta_fit + 1) - t1_val ** (beta_fit + 1)) / (
    beta_fit + 1
) + gd_offset_fit * (t2_val - t1_val)

sinc_arg_fit = 0.5 * q * h * Gamma_fit * np.cos(phi_fine_rad - phi0_fit_rad)
sinc_sq_fit = sinc(sinc_arg_fit) ** 2
c2_model_phi = 1.0 + 0.12 * np.exp(-2 * q**2 * J_fit) * sinc_sq_fit

# Experimental values at this lag
c2_exp_phi = np.array(
    [
        np.mean([c2[i_phi, k, k + lag_idx] for k in range(n_t - lag_idx)])
        for i_phi in range(n_phi)
    ]
)

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(phi_fine, c2_model_phi, "r-", linewidth=2, label="Model (fitted)")
ax.plot(phi_deg, c2_exp_phi, "ko", markersize=6, label="Experiment")
ax.axvline(
    TRUE["phi_0"],
    color="blue",
    linestyle="--",
    alpha=0.5,
    label=f"True phi_0={TRUE['phi_0']}°",
)
ax.axvline(
    phi0_fit % 360,
    color="red",
    linestyle=":",
    alpha=0.7,
    label=f"Fitted phi_0={phi0_fit:.1f}°",
)
ax.set_xlabel("Azimuthal angle phi (degrees)")
ax.set_ylabel("C2")
ax.set_title(
    f"Angular dependence at lag = {lag_idx * dt:.1f} s\n(sinc² pattern from shear)"
)
ax.legend()
ax.set_xlim(0, 360)
plt.tight_layout()
display(fig)
plt.close(fig)

print(f"True phi_0:   {TRUE['phi_0']:.1f}°")
print(f"Fitted phi_0: {phi0_fit:.1f}°")
print(f"Difference:   {abs(phi0_fit - TRUE['phi_0']):.2f}°")

## 7. Physical Interpretation of Shear Parameters

In [ ]:
applied_shear_rate = 2.0  # s⁻¹ (from rheometer)
fitted_shear_rate = result_flow.parameters[phys_offset + 3]
fitted_shear_err = result_flow.uncertainties[phys_offset + 3]

print("Shear Parameter Analysis")
print("=" * 40)
print(f"Applied shear rate:  {applied_shear_rate:.3f} s⁻¹")
print(f"XPCS fitted rate:    {fitted_shear_rate:.3f} ± {fitted_shear_err:.3f} s⁻¹")
print(f"Ratio (XPCS/applied): {fitted_shear_rate / applied_shear_rate:.2f}")
print()

beta_fit = result_flow.parameters[phys_offset + 4]
if abs(beta_fit) < 0.1:
    print(f"beta = {beta_fit:.3f} → approximately steady shear")
elif beta_fit < 0:
    print(
        f"beta = {beta_fit:.3f} → shear rate decreasing with time (shear thinning or startup)"
    )
else:
    print(f"beta = {beta_fit:.3f} → shear rate increasing with time (shear thickening)")

phi0_fit = result_flow.parameters[phys_offset + 6]
print(f"\nFlow direction (phi_0): {phi0_fit:.1f}°")
print("(0° = flow direction aligned with horizontal detector axis)")

## 8. Summary

Key takeaways from laminar flow analysis:

- **Angular dependence** in C2 is the diagnostic for laminar flow
- **`per_angle_mode: "auto"`** is essential to prevent parameter absorption degeneracy
- **`phi_0`** tells you the flow direction relative to the detector
- **`gamma_dot_0`** should match the applied rheometer shear rate (within ~10%)
- Deviations between XPCS and rheometer shear rates may indicate slip or inhomogeneous flow

For Bayesian uncertainty quantification of laminar flow parameters, see `04_bayesian_inference.ipynb`.

In [ ]:
os.unlink(config_path)